In [30]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from IPython.display import clear_output,display


In [31]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

import time

In [38]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")
user_id = new_user['id']
user_id
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)
meeting_url="https://meet.google.com/iya-faxw-xze"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="Vexa",
  #  language='es',
    task = 'transcribe'
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '5308@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 06 Jun 2025 09:49:56 GMT, Fri, 06 Jun 2025 09:49:57 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '133', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"5308@example.com","name":"test","image_url":null,"max_concurrent_bots":1,"id":18,"created_at":"2025-06-06T09:49:57.456777"}...

DEBUG: Making POST request to http://localhost:18056/admin/users/18/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 06 Jun 2025 09:49:56 GMT, Fri, 06 Jun 2025 09:49:57 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '115', 'content-

In [51]:
while True:
    transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
    df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
    clear_output()
    display(df.sort_values('absolute_start_time').tail(10))
    time.sleep(1)

KeyError: 'absolute_start_time'

In [17]:
client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/iya-faxw-xze
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'DnmtBJ45zZk7fCDNMVRnihydxvJYBwn5WKQq1sLs'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Fri, 06 Jun 2025 09:44:46 GMT, Fri, 06 Jun 2025 09:44:46 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...


{'message': 'Stop request accepted and is being processed.'}

In [ ]:
while True:
    transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
    df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
    clear_output()
    display(df.sort_values('absolute_start_time').tail(10))
    time.sleep(1)

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,0.50,Oh.,None,None,Dmitriy Grankin,2025-06-04T18:58:38.372716+00:00,2025-06-04T18:58:38.872716+00:00
1,0.50,12.16,"And hello, how is it going?",None,None,Dmitriy Grankin,2025-06-04T18:58:38.872716+00:00,2025-06-04T18:58:50.532716+00:00
2,0.80,1.30,Ahem.,None,2025-06-04T18:59:12.761410,Dmitriy Grankin,2025-06-04T18:58:39.172716+00:00,2025-06-04T18:58:39.672716+00:00
3,12.16,13.76,Where is the transcript?,None,None,Dmitriy Grankin,2025-06-04T18:58:50.532716+00:00,2025-06-04T18:58:52.132716+00:00
4,18.11,28.87,okay and now I will evict this one,None,None,Dmitriy Grankin,2025-06-04T18:58:56.483716+00:00,2025-06-04T18:59:07.243716+00:00
5,18.29,28.87,okay and now I will evict this one,None,None,Dmitriy Grankin,2025-06-04T18:58:56.663716+00:00,2025-06-04T18:59:07.243716+00:00


KeyboardInterrupt: 

In [8]:
user_id

2

In [309]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '5IRfzrEVsU2T8VBq0oNIFA1uIddTbAlUiiFaesTq'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 27 May 2025 22:35:23 GMT, Tue, 27 May 2025 22:35:23 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '19', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[]}...


In [32]:
admin_client.update_user(user_id,max_concurrent_bots=3)


DEBUG: Making PATCH request to http://localhost:8056/admin/users/2
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'lkwnekf68&IJBKb7'}
DEBUG: Params: None
DEBUG: JSON data: {'max_concurrent_bots': 3}
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:27:45 GMT, Tue, 22 Apr 2025 13:27:45 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '134', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"914545@example.com","name":"test","image_url":null,"max_concurrent_bots":3,"id":2,"created_at":"2025-04-22T13:17:30.891747"}...


{'email': '914545@example.com',
 'name': 'test',
 'image_url': None,
 'max_concurrent_bots': 3,
 'id': 2,
 'created_at': '2025-04-22T13:17:30.891747'}

In [49]:
client.update_bot_config(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    language='en',
    task = 'transcribe'
)


DEBUG: Making PUT request to http://localhost:18056/bots/google_meet/icj-aejh-tac/config
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'VLZbS0Vh3nNBNk8woojDWBG1r56cwP5PEIdPM74s'}
DEBUG: Params: None
DEBUG: JSON data: {'language': 'en', 'task': 'transcribe'}
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 17:48:37 GMT, Wed, 04 Jun 2025 17:48:38 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Reconfiguration request accepted and sent to the bot."}...


{'message': 'Reconfiguration request accepted and sent to the bot.'}

In [ ]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:8056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'yl9x4xUZdXnLZsRxfj6A7IQJEyf2czFpQBjV1e93'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:58:29 GMT, Tue, 22 Apr 2025 13:58:29 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1202', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"e9c6def1ea1468b67b5bb8c010eff03febadf40e8b1727d6dadf8a72ddeb9c9a","container_name":"vexa-bot-11-4f4fd204","platform":"google_meet","native_meeting_id":"phu-przh-ove","status":"Up 30 minutes","created_at":"2025-04-22T13:27:49+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"2"},"meeting_id_from_name":"11"},{"container_id":"433f2e1fd9776f6ddae1262b530b62b66b5a3bca16be9b25bfe1a7b10f3c3967","con...


In [ ]:
running_bots = client.get_running_bots_status()
for running_bot in running_bots:
    client.stop_bot(platform=running_bot['platform'],native_meeting_id=running_bot['native_meeting_id'])


DEBUG: Making GET request to http://localhost:8056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'yl9x4xUZdXnLZsRxfj6A7IQJEyf2czFpQBjV1e93'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:58:29 GMT, Tue, 22 Apr 2025 13:58:29 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1202', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"e9c6def1ea1468b67b5bb8c010eff03febadf40e8b1727d6dadf8a72ddeb9c9a","container_name":"vexa-bot-11-4f4fd204","platform":"google_meet","native_meeting_id":"phu-przh-ove","status":"Up 30 minutes","created_at":"2025-04-22T13:27:49+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"2"},"meeting_id_from_name":"11"},{"container_id":"433f2e1fd9776f6ddae1262b530b62b66b5a3bca16be9b25bfe1a7b10f3c3967","con...


In [298]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:8056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'NvsJRMspflhJ8qRZ5IQiZDrfkdsXyqncuaVlvKoR'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Mon, 21 Apr 2025 15:49:02 GMT, Mon, 21 Apr 2025 15:49:02 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '406', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":6,"user_id":2,"platform":"google_meet","native_meeting_id":"fsn-apcm-nqf","constructed_meeting_url":"https://meet.google.com/fsn-apcm-nqf","status":"active","bot_container_id":"71d0469f81097eb209f27ab3ecfad10f80c93bf010eb9a8ed982f01122eda6ac","start_time":"2025-04-21T15:48:34.986970","end_time":null,"created_at":"2025-04-21T15:48:34.750718","updated_at":"2025-04-21T15:48:34.757661"}]}...
